In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

import numpy as np
import SimpleITK as sitk

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [4]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [5]:
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'

csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'

In [8]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [9]:
dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

In [10]:
print(len(dataset))


617


In [ ]:
#GENERE LA LISTE DES JSON AVEC UN UNCONSTANT SPACING, SAVE IT AS JSON 

In [ ]:

from library_dicom.dicom_processor.tools.folders import *
liste_unconstant_z_spacing = []
for liste in dataset : 
    print(dataset.index(liste))
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            print(liste)
            liste_unconstant_z_spacing.append(liste)
            list_z_spacing = serie_objet.calculate_z_spacing()
            print(list_z_spacing)
            #liste_unconstant_z_spacing.append(list_z_spacing)


#save it as json

write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
#On enleve les series avec un unconstant spacing 
for serie in liste_unconstant_z_spacing : 
    dataset.remove(serie)

In [ ]:
#SAVE AS JSON THE FINAL LIST WITHOUT UNCONSTANT SPACING ERROR

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_final_list_dataset', dataset)


In [ ]:
print(len(dataset)) #589


In [11]:
#RECHARGEZ LE JSON AVEC LES SERIES 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final_list_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

589


In [ ]:
#IF MASK IS CORRECT => GENERATE NIFTI 
#IF PHILIPS, OR MASK FALSE => PUT SERIE IN A NEW LISTE 

In [ ]:
untreated_liste = []
#CT et PT pas dans le même ordre suivant les séries 
#voir pour les ranger avant PT en premier puis CT 


#error = []

#trier les philips des autres series fausses ici 


for serie in dataset : 
    try : 
        print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                     #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
                   


    except Exception as err : 
        print(err)
        #error.append(serie)

        


In [ ]:
len(untreated_liste) #392 

In [ ]:
#SAVE THE SERIE WITH FALSE MASK 
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_false_mask', untreated_liste)
print(len(untreated_liste))

In [ ]:
#SEARCH SERIE WITH ERROR ET SAVE IT AS JSON 

In [12]:
#IMPORTER JSON OF FINAL LISTE WITHOUT UNCONSTANT SPACING PROBLEM 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final_list_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))


589


In [16]:
#sauver les séries qui font des erreurs dans un autre json 
serie_with_error = []
for serie in dataset : 
    try : 
        #print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

    except Exception as err : 
        print(serie)
        print(err)
        serie_with_error.append(serie)

from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_error', serie_with_error)
#On traitera après 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101021022/1.2.250.1.59.470.940100027.1.20130802113038.1925.124326/1.3.46.670589.28.2.10.4.9167.27921.2.460.0.1376045065/1.3.46.670589.28.2.10.4.9167.27921.2.460.0.1376045065', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101021022/1.2.250.1.59.470.940100027.1.20130802113038.1925.124326/1.2.840.113704.1.111.2068.1376042666.8/1.2.840.113704.1.111.2068.1376042666.8', 'CT', 'mus_ar_aug 9_2013_abr.csv']
float division by zero
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
1

In [18]:
len(serie_with_error)
serie_with_error.append(dataset[220])
serie_with_error.append(dataset[231])
serie_with_error.append(dataset[325])
serie_with_error.append(dataset[474])
serie_with_error.append(dataset[453])
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_error', serie_with_error)
# => 22 séries avec des erreurs, à voir 

In [ ]:
220
231
325
474

Error generating result array time

 data '' does not match format '%Y%m%d%H%M%S'

453
Error generating result array unsupported operand type(s) for *: 'NoneType' and 'int'

#float division par 0
#index 255 is out of bounds for axis 0 with size 255

In [ ]:
#IMPORT SERIES WITH FALSE MASK,
#CHECK FLIP AXE Z + SMALL ERROR ON ELLIPSE 

In [13]:
#importer la liste untreated_serie pour la suite : 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_serie_with_false_mask.json'
untreated_liste = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        untreated_liste.append(info)

print(len(untreated_liste)) #392

392


In [14]:
#SEPARE LES SERIES PHILIPS DES AUTRES SERIES UNTREATED
philips_serie = []
untreated_liste_2 = []
for serie in untreated_liste : 
    #print(untreated_liste.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                philips_serie.append(serie)
            else : untreated_liste_2.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                philips_serie.append(serie)
            else : untreated_liste_2.append(serie)

    except Exception as err : 
        print(serie)
        print(err)


print(len(philips_serie))
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_untreated_philips_serie', philips_serie)
print(len(untreated_liste_2))
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_untreated_serie', untreated_liste_2)

203
189


In [15]:
#IMPORT JSON (serie with false mask, except Philips)
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_untreated_serie.json'
untreated_liste_2 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        untreated_liste_2.append(info)

print(len(untreated_liste_2)) #189

189


In [16]:
#SI PETITE ERREUR SUR MEAN/SD  SANS LE FLIP, ON TOLERE = > Genere nifti 
#error mask = problème sur les SUV MAX, ou problème SUV MEAN/SD SUR LES POLYGONES = > A CHECKER
serie_error_mask = []
for serie in untreated_liste_2 : 
    print(untreated_liste_2.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if mask_objet.ecart_suv_max(nifti_array) != [] :
                print("PROBLEME SUV MAX")
                serie_error_mask.append(serie)
            else : 
                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : #sinon problème sur les MAX A CHECK
                    serie_error_mask.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if mask_objet.ecart_suv_max(nifti_array) != [] :
                print("PROBLEME SUV MAX")
                serie_error_mask.append(serie)
            else : 
                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : #sinon problème sur les MAX A CHECK
                    serie_error_mask.append(serie)


    except Exception as err : 
        print(serie)
        print(err)



0
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
1
PROBLEME SUV MAX
2
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
3
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
4
PROBLEME SUV MAX
5
6
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
7
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
8
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
9
PROBLEME SUV MAX
10
11
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
12
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
13
PROBLEME SUV MAX
14
PROBLEME SUV MAX
15
PROBLEME SUV MAX
16
PROBLEME SUV MAX
17
PROBLEME SUV MAX
18
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
19
SMALL ERRORS ON ELLIPS

In [ ]:
#save as json the list of serie which have a mask problem 
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_error_mask', serie_error_mask)

In [ ]:
#CHECKER LE FLIP : AUCUNE SERIE NE MARCHAIT EN FAISANT UN FLIP 

In [9]:
#ON FLIP LE MASK ET ON REGARDE SI LES CALCLS SUV MAX MEAN SD SONT BONS
#SI OUI => GENERE NIFTI 

serie_with_flip = []
for serie in untreated_liste_2 : 
    print(untreated_liste_2.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if (mask_objet.is_correct_suv(nifti_array, flip = True) == True) : #si mask correct APRES LE FLIP 
                print("MASK CORRECT APRES FLIP")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                #generation nifti mask
                mask_objet.flip_z(mask_4D)
                mask_4D_flip = mask_objet.mask_array
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                print("EXPORT NIFTI MASK")
                serie_with_flip.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if (mask_objet.is_correct_suv(nifti_array, flip = True) == True) : #si mask correct APRES LE FLIP 
                print("MASK CORRECT APRES FLIP")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                #generation nifti mask
                mask_objet.flip_z(mask_4D)
                mask_4D_flip = mask_objet.mask_array
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                print("EXPORT NIFTI MASK")
                serie_with_flip.append(serie)



    except Exception as err : 
        print(serie)
        print(err)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188


In [ ]:
print(len(untreated_liste_2))

In [ ]:
#IMPORT JSON SERIES PHILIPS
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_philips_serie.json'
philips_serie = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        philips_serie.append(info)

print(len(philips_serie)) #203

In [ ]:
#regarder les erreurs de max autres que philips 
#regarder les erreurs philips 
#problème de datetime dans certaines séries : 
        #Error generating result array time data '' does not match format '%Y%m%d%H%M%S'

In [ ]:
#philips -> MIP MASK 